# Rank-1 CNN on MNIST dataset

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
import re
import os

#import MNIST #
from tensorflow.examples.tutorials.mnist import input_data

#save the MNIST data in to the folder /mnist/datat/
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

# to plot images import the necessary libraries        
#%pylab      

# use matplotlib as inline
%matplotlib inline 

from matplotlib.pyplot import *
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allocator_type ='BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.90

In [3]:
# conv1_1
x = tf.placeholder("float", [None, 28, 28, 1])
phase = tf.placeholder(tf.bool, name='phase') #batch normalization을 training모드인지 test모드인지

input_num = 1 #number of input channels
dim = 3 # Size of the 1-D vectors
num = 8 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list1_1 = [] # List for W

P1_1 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q1_1 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
T1_1 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))

k=0
for i in range(num): 
    for j in range(num):
      P1_1_temp = tf.reshape(P1_1[i], [dim, 1])
      Q1_1_temp = tf.reshape(Q1_1[j], [1, dim])
      Mul_temp1_1_a = tf.matmul(P1_1_temp, Q1_1_temp)
      Mul_temp1_1_b = tf.reshape(Mul_temp1_1_a, [1, Mul_temp1_1_a.shape[0], Mul_temp1_1_a.shape[1]])
      Mul_temp1_1_c = T1_1[k]*Mul_temp1_1_b
      k = k + 1
      W_list1_1.append(Mul_temp1_1_c)

W1_1 = tf.transpose(tf.stack(W_list1_1, axis=0))
L1 = tf.nn.conv2d(x, W1_1, strides=[1, 1, 1, 1], padding='SAME')

In [6]:
# conv1_2

input_num = 64 #number of input channels
dim = 3 # Size of the 1-D vectors
num = 8 # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list1_2 = [] # List for W

P1_2 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q1_2 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
T1_2 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))

k=0
for i in range(num): 
    for j in range(num):
      P1_2_temp = tf.reshape(P1_2[i], [dim, 1])
      Q1_2_temp = tf.reshape(Q1_2[j], [1, dim])
      Mul_temp1_2_a = tf.matmul(P1_2_temp, Q1_2_temp)
      Mul_temp1_2_b = tf.reshape(Mul_temp1_2_a, [1, Mul_temp1_2_a.shape[0], Mul_temp1_2_a.shape[1]])
      Mul_temp1_2_c = T1_2[k]*Mul_temp1_2_b
      k = k + 1
      W_list1_2.append(Mul_temp1_2_c)

W1_2 = tf.transpose(tf.stack(W_list1_2, axis=0))
L1 = tf.nn.conv2d(L1, W1_2, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [11]:
# conv2_1

input_num = 64 #number of input channels
dim = 3   # Size of the 1-D vectors
num = 12  # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list2_1 = [] # List for W

P2_1 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q2_1 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))

T2_1 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))
k=0
for i in range(num): 
    for j in range(num):
      #P_transpose = tf.reshape(P[:,0], [dim, 1]) --> 이렇게 해도 되지만....
      #P는 row가 column이다라고 기억하고서 다음과 같이 하자...
      P2_1_temp = tf.reshape(P2_1[i], [dim, 1])
      Q2_1_temp = tf.reshape(Q2_1[j], [1, dim])
      Mul_temp2_1_a = tf.matmul(P2_1_temp, Q2_1_temp)
      Mul_temp2_1_b = tf.reshape(Mul_temp2_1_a, [1, Mul_temp2_1_a.shape[0], Mul_temp2_1_a.shape[1]])
      Mul_temp2_1_c = T2_1[k]*Mul_temp2_1_b
      k = k + 1
      W_list2_1.append(Mul_temp2_1_c)

W2_1 = tf.transpose(tf.stack(W_list2_1, axis=0))
L2 = tf.nn.conv2d(L1, W2_1, strides=[1, 1, 1, 1], padding='SAME')

In [15]:
# conv2_2

input_num = 144  #number of input channels
dim = 3   # Size of the 1-D vectors
num = 12  # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list2_2 = [] # List for W

P2_2 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q2_2 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
T2_2 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))

k=0
for i in range(num): 
    for j in range(num):
      P2_2_temp = tf.reshape(P2_2[i], [dim, 1])
      Q2_2_temp = tf.reshape(Q2_2[j], [1, dim])
      Mul_temp2_2_a = tf.matmul(P2_2_temp, Q2_2_temp)
      Mul_temp2_2_b = tf.reshape(Mul_temp2_2_a, [1, Mul_temp2_2_a.shape[0], Mul_temp2_2_a.shape[1]])
      Mul_temp2_2_c = T2_2[k]*Mul_temp2_2_b
      k = k + 1
      W_list2_2.append(Mul_temp2_2_c)

W2_2 = tf.transpose(tf.stack(W_list2_2, axis=0))
L2 = tf.nn.conv2d(L2, W2_2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [20]:
# conv3_1

input_num = 144  #number of input channels
dim = 3    # Size of the 1-D vectors
num = 16   # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list3_1 = [] # List for W

P3_1 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q3_1 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
T3_1 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))

k=0
for i in range(num): 
    for j in range(num):
      P3_1_temp = tf.reshape(P3_1[i], [dim, 1])
      Q3_1_temp = tf.reshape(Q3_1[j], [1, dim])
      Mul_temp3_1_a = tf.matmul(P3_1_temp, Q3_1_temp)
      Mul_temp3_1_b = tf.reshape(Mul_temp3_1_a, [1, Mul_temp3_1_a.shape[0], Mul_temp3_1_a.shape[1]])
      Mul_temp3_1_c = T3_1[k]*Mul_temp3_1_b
      k = k + 1
      W_list3_1.append(Mul_temp3_1_c)

W3_1 = tf.transpose(tf.stack(W_list3_1, axis=0))
L3 = tf.nn.conv2d(L2, W3_1, strides=[1, 1, 1, 1], padding='SAME')

In [22]:
# conv3_2

input_num = 256   #number of input channels
dim = 3   # Size of the 1-D vectors
num = 16  # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list3_2 = [] # List for W

P3_2 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q3_2 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
T3_2 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))

k=0
for i in range(num): 
    for j in range(num):
      P3_2_temp = tf.reshape(P3_2[i], [dim, 1])
      Q3_2_temp = tf.reshape(Q3_2[j], [1, dim])
      Mul_temp3_2_a = tf.matmul(P3_2_temp, Q3_2_temp)
      Mul_temp3_2_b = tf.reshape(Mul_temp3_2_a, [1, Mul_temp3_2_a.shape[0], Mul_temp3_2_a.shape[1]])
      Mul_temp3_2_c = T3_2[k]*Mul_temp3_2_b
      k = k + 1
      W_list3_2.append(Mul_temp3_2_c)

W3_2 = tf.transpose(tf.stack(W_list3_2, axis=0))
L3 = tf.nn.conv2d(L3, W3_2, strides=[1, 1, 1, 1], padding='SAME')

In [24]:
# conv3_3

input_num = 256  #number of input channels
dim = 3   # Size of the 1-D vectors
num = 16  # Number of P and Q vectors. The combination of P and Q vectors becomes num * num
output_num = num*num  # number of filters = number of outputs
W_list3_3 = [] # List for W

P3_3 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
Q3_3 = tf.Variable(tf.random_uniform([num,dim], -0.5, 0.5))
T3_3 = tf.Variable(tf.random_uniform([output_num,input_num,1,1], -0.5, 0.5))

k=0
for i in range(num): 
    for j in range(num):
      P3_3_temp = tf.reshape(P3_3[i], [dim, 1])
      Q3_3_temp = tf.reshape(Q3_3[j], [1, dim])
      Mul_temp3_3_a = tf.matmul(P3_3_temp, Q3_3_temp)
      Mul_temp3_3_b = tf.reshape(Mul_temp3_3_a, [1, Mul_temp3_3_a.shape[0], Mul_temp3_3_a.shape[1]])
      Mul_temp3_3_c = T3_3[k]*Mul_temp3_3_b
      k = k + 1
      W_list3_3.append(Mul_temp3_3_c)

W3_3 = tf.transpose(tf.stack(W_list3_3, axis=0))
L3 = tf.nn.conv2d(L3, W3_3, strides=[1, 1, 1, 1], padding='SAME')

In [27]:
def dense(inputs, units, name=None): 
#"""3x3 conv layer: ReLU + He initialization""" 
# He initialization: normal dist with stdev = sqrt(2.0/fan-in) 
    stddev = np.sqrt(2 / int(inputs.shape[1])) 
    out = tf.layers.dense(inputs, units, 
                           kernel_initializer=tf.random_normal_initializer(stddev=stddev), 
                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1.0), 
                           name=name) 

    return out 

In [28]:
def dense_relu(inputs, units, name=None): 
# """3x3 conv layer: ReLU + He initialization""" 
 
 
# He initialization: normal dist with stdev = sqrt(2.0/fan-in) 
    stddev = np.sqrt(2 / int(inputs.shape[1])) 
    out = tf.layers.dense(inputs, units, activation=tf.nn.relu, 
                         kernel_initializer=tf.random_normal_initializer(stddev=stddev), 
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(1.0), 
                         name=name) 
 
 
#   tf.summary.histogram('act' + name, out) 

 
    return out 

In [29]:
def dense_batch_relu(x, output, phase, scope):
    with tf.variable_scope(scope):
        h1 = tf.contrib.layers.fully_connected(x, output, 
                                               activation_fn=None,
                                               scope='dense')
        h2 = tf.contrib.layers.batch_norm(h1, 
                                          center=True, scale=True, 
                                          is_training=phase,
                                          scope='bn')
        return tf.nn.relu(h2, 'relu')

In [30]:
y_ = tf.placeholder("float", [None,10])

In [32]:
# fully connected layers
out = tf.contrib.layers.flatten(L3)
out = dense_batch_relu(out, 2048, phase,'fc1')
out = tf.nn.dropout(out, 0.5) 
out = dense_batch_relu(out, 1024, phase,'fc2')
out = tf.nn.dropout(out, 0.5) 

class_number = 10
logits = dense(out, class_number, 'fc3')  # class number =2

In [34]:
#########
# Result
######
is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [35]:
# Loss 함수
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))

saver = tf.train.Saver(tf.global_variables())
global_step = tf.Variable(0, trainable=False, name='global_step')

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
     train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost, global_step=global_step)
        
# variable initializer
init = tf.global_variables_initializer()

#sess = tf.Session()
sess = tf.InteractiveSession()
sess.run(init)

batch_size_train = 100
batch_size_test = 100   
total_train_batch = int(mnist.train.num_examples / batch_size_train)
total_test_batch = int(mnist.test.num_examples / batch_size_test)

for epoch in range(100):
    total_cost = 0
    total_accuracy = 0

    start_time = time.time()
    for i in range(total_train_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size_train)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([train_step, cost],
                               feed_dict={x: batch_xs,y_: batch_ys, phase: True})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_train_batch))
    print("--- %s seconds ---" % (time.time() - start_time))
        
    for i in range(total_test_batch):
        batch_xs2, batch_ys2 = mnist.test.next_batch(batch_size_test)   
        batch_xs2 = batch_xs2.reshape(-1, 28, 28, 1) 
        acc = sess.run(accuracy, feed_dict={x: batch_xs2, y_: batch_ys2, phase: False})
        total_accuracy += acc    
    
    mean_accuracy = total_accuracy/total_test_batch
    print('Accuracy:', mean_accuracy) 
          
print('End of optimization!')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch: 0001 Avg. cost = 0.287
--- 116.9731342792511 seconds ---
Accuracy: 0.35869999960064886
Epoch: 0002 Avg. cost = 0.119
--- 99.87699961662292 seconds ---
Accuracy: 0.44809999942779544
Epoch: 0003 Avg. cost = 0.092
--- 100.00199508666992 seconds ---
Accuracy: 0.5119999957084655
Epoch: 0004 Avg. cost = 0.077
--- 99.8930037021637 seconds ---
Accuracy: 0.6080000001192093
Epoch: 0005 Avg. cost = 0.068
--- 100.19200325012207 seconds ---
Accuracy: 0.7151000028848649
Epoch: 0006 Avg. cost = 0.059
--- 100.26400899887085 seconds ---
Accuracy: 0.869699998497963
Epoch: 0007 Avg. cost = 0.055
--- 100.29199981689453 seconds ---
Accuracy: 0.928500000834465
Epoch: 0008 Avg. cost = 0.049
--- 99.72399997711182 seconds ---
Accuracy: 0.9659000062942504
Epoch: 0009 Avg. cost = 0.047
--- 99.83499789237976 seconds -